# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3, MovieMaker
from tqdm import tqdm

url = 'localhost:5000'
be = BioExplorer(url)
mm = MovieMaker(be)
core = be.core_api()

load_cells = True
load_meshes = False
cell_radius = 5.0 # Micrometters

nissl_enabled = False


### Load Atlas

In [ ]:
# population_name = 'atlas_ccfv3a_averaged'
population_name = 'atlas_ccfv3a'
if nissl_enabled:
    population_name = 'atlas_ccfv3a_all_cells'
    cell_radius = 1.0 # Micrometters

In [ ]:
fibers_ids = [960, 1000, 1009, 396, 109, 698]
regions_to_ignore = [
    
    1557651848, # Cell excluded from region 1557651847 (x<540)
    1024543562, 2358040414, 3263488087, 2416897036,
    3034756217, 2165415682, 2614168502, 1842735199,
    3101970431, 1140764290, 3092369320, 1811993763]

In [ ]:
ignore_filter = str(fibers_ids + regions_to_ignore).replace('[','').replace(']','')

In [ ]:
status = be.reset_scene()

In [ ]:
atlas_assembly_name = 'Cell Atlas'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=load_cells, load_meshes=load_meshes, cell_radius=cell_radius,
    region_sql_filter='guid NOT IN (%s)' % ignore_filter,
    mesh_scale=Vector3(1, 1, 1)
)

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
region_colors = dict()

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) / 256.0 for i in range(0, lv, lv // 3))

with Session(engine) as session:
    data = session.execute('select guid, color_hex_triplet from %s.region' % population_name)
    for d in data.all():
        region_colors[int(d[0])] = hex_to_rgb(d[1])

In [ ]:
model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    k = 0
    material_ids = be.get_material_ids(model_id)['ids']
    palette = list()
    opacities = list()
    shading_modes = list()
    specular_exponents = list()
    refraction_indices = list()
    glossinesses = list()
    for material_id in material_ids:
        if nissl_enabled:
            c = [0.8, 0.3, 0.8]
        else:
            c = [1, 0, 0]
            m_id = material_id
            if m_id in region_colors:
                c = region_colors[m_id]
        alpha = [1.0, 1.0, 1.0]

        shading_modes.append(be.shading_mode.NONE)
        glossinesses.append(1.0)
        specular_exponents.append(50.0)
        palette.append([alpha[0] * c[0], alpha[1] * c[1], alpha[2] * c[2]])
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, 
        opacities=opacities, specular_exponents=specular_exponents,
        refraction_indices=refraction_indices, glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette)

In [ ]:
status = core.clear_lights()
core.add_light_directional(
    angularDiameter=45, color=(1,1,1), direction=(0.5, 0.5, 1.2),
    intensity=1.0, is_visible=False)

In [ ]:
background_color = [0.0, 0.0, 0.0]
if nissl_enabled:
    background_color = [1.0, 1.0, 1.0]

status = core.set_renderer(
    head_light=False,
    current='advanced', background_color=background_color,
    subsampling=4, max_accum_frames=128)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1e6
params.shadow_intensity = 1.0
params.soft_shadow_strength = 0.5
params.main_exposure = 1.5
params.max_ray_depth = 1
params.epsilon_multiplier = 50.0
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

In [ ]:
core.set_camera(
    current='perspective',
    orientation=[0.8744458160476791, 0.09151682620478456, -0.401564627390352, 0.25634943991934056],
    position=[-2565.3701241987646, -2607.8377454106976, -1731.3329308640486],
    target=[6733.589672442965, 4796.273454159725, 6016.635720470601],
)

### Save to cache

In [ ]:
from bioexplorer import SonataExplorer
se = SonataExplorer(be)

In [ ]:
se.save_model_to_cache(model_id=1, path='ccfv3a.soc')